#  importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, auc
import seaborn as sns
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_auc_score

# Function to load images from a folder

In [ ]:
def load_images_from_folder(folder):
    images = []
    labels = []

    for subfolder in os.listdir(folder):
        subfolder_path = os.path.join(folder, subfolder)

        if os.path.isdir(subfolder_path):
            label = subfolder
            for filename in os.listdir(subfolder_path):
                img_path = os.path.join(subfolder_path, filename)
                img = cv2.imread((img_path), cv2.IMREAD_GRAYSCALE)

                if img is not None:
                    img = cv2.resize(img, (224, 224))  # Resize image to 224x224 pixels
                    images.append(img)
                    labels.append(label)

    return np.array(images), np.array(labels)


# Load images and labels

In [ ]:
path = "F:/dataset/"
images, labels = load_images_from_folder(path)

flat_images = np.array(images)
target = np.array(labels)


# Flatten images

In [ ]:
l = []
for i in range(len(flat_images)):
    l.append(flat_images[i].flatten())

flat_images_flat = np.array(l)


# Create a DataFrame

In [ ]:
df = pd.DataFrame(flat_images_flat)
target = np.array(labels)
df['Target'] = target

# Split the dataset into the Training set and Test set

In [ ]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)


# FITTING

In [ ]:
log = LogisticRegression()
log.fit(X_train, y_train)
y_pred = log.predict(X_test)


# Calculate accuracy

In [ ]:
accuracy_no_hog = accuracy_score(y_test, y_pred)
print(f"Accuracy without HOG: {accuracy_no_hog}")


#  Function to preprocess images for HOG

In [ ]:
def preprocess_images_for_hog(images, size=(32, 32)):
    processed_images = []
    for img in images:
        resized_img = cv2.resize(img, size, interpolation=cv2.INTER_AREA)
        if resized_img.shape[-1] == 3:
            grayscale_img = cv2.cvtColor(resized_img, cv2.COLOR_BGR2GRAY)
        else:
            grayscale_img = resized_img
        processed_images.append(grayscale_img)
    return np.array(processed_images)

#  Preprocess images for HOG

In [ ]:
train_images_processed = preprocess_images_for_hog(images)

 #  Compute HOG features for Training images

In [ ]:
train_hog_features = []
for img in train_images_processed:
    hog_img = hog(img, visualize=False)
    train_hog_features.append(hog_img)


#  Split the dataset with HOG features

In [ ]:
X_train_processed, X_test_processed, y_train_processed, y_test_processed = train_test_split(train_hog_features, labels, test_size=0.2, random_state=10)

#  Train the model with HOG features

In [ ]:
log_hog = LogisticRegression()
log_hog.fit(X_train_processed, y_train_processed)
y_pred_hog = log_hog.predict(X_test_processed)


#  Calculate accuracy with HOG

In [ ]:
accuracy_hog = accuracy_score(y_test_processed, y_pred_hog)
print(f"Accuracy with HOG: {accuracy_hog}")


# Create confusion matrix for logistic regression with HOG features

In [ ]:
conf_matrix_hog = confusion_matrix(y_test_processed, y_pred_hog)
sns.heatmap(conf_matrix_hog, annot=True, fmt='d', cmap='Blues', xticklabels=np.unique(y_test_processed), yticklabels=np.unique(y_test_processed))
plt.title('Confusion Matrix with HOG')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()


# create ROC CURVE


In [ ]:
y_test_bin = label_binarize(y_test_processed, classes=np.unique(y_test_processed))

# Use decision function instead of predict_proba for Logistic Regression

In [ ]:
y_score_hog = log_hog.decision_function(X_test_processed)

 #   Compute ROC curve and ROC area for each class

In [ ]:
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(len(np.unique(y_test_processed))):
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_score_hog[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot ROC curve

In [ ]:
plt.figure(figsize=(8, 6))
for i in range(len(np.unique(y_test_processed))):
    plt.plot(fpr[i], tpr[i], label=f'Class {i} (AUC = {roc_auc[i]:.2f})')

plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for Logistic Regression with HOG')
plt.legend(loc="lower right")
plt.show()

#  loss curve


In [ ]:
log_hog = LogisticRegression(max_iter=100)

accuracies = []

# Training loop with accuracy collection
for iteration in range(log_hog.max_iter):
    log_hog.fit(X_train_processed, y_train_processed)
    accuracy = log_hog.score(X_train_processed, y_train_processed)
    accuracies.append(accuracy)
    print(f"Iteration {iteration + 1}/{log_hog.max_iter} - Accuracy: {accuracy}")

# Plot the accuracy curve
plt.figure(figsize=(8, 6))
plt.plot(accuracies, label='Training Accuracy')
plt.xlabel('Iteration')
plt.ylabel('Accuracy')
plt.title('Training Accuracy Curve')
plt.legend()
plt.show()